In [3]:
from datasets import load_dataset 
load_dataset("jnlpba")

README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

jnlpba.py:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3856 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 18546
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3856
    })
})

In [1]:
from model import lora
from torchvision import models


resnet18 = models.resnet18(pretrained=True)
print(resnet18)
lora_resnet18 = lora.Lora(resnet18)
lora_resnet18._apply_lora(lora_resnet18.model.fc)
print(lora_resnet18)


/home/zhu.3723/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zhu.3723/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/zhu.3723/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
import torch 
input_x = torch.randn(1, 3, 224, 224)
output = lora_resnet18(input_x)
print(output)

pseudo_label tensor([[-0.5910, -0.3493, -0.6492, -1.5522, -0.7644, -0.2025, -0.5300,  0.5564,
          0.4337, -0.6773, -1.0394, -0.9122, -0.3085, -0.9754, -1.1184, -0.5930,
         -0.9025, -0.2865, -0.5246, -0.5870, -1.5653, -0.7951, -1.3831,  0.1732,
         -0.9422, -1.0763, -0.7888, -1.1063, -0.8880, -0.3786, -0.9136, -0.8250,
         -0.5466, -0.5231, -0.3847, -0.4555,  0.5807, -0.7896, -0.5481, -0.0846,
         -0.7958, -0.9499, -1.0525, -0.4302, -0.7772, -0.4823, -0.8727, -0.7115,
         -1.2726, -1.0723, -0.4903,  0.4098, -0.4022, -0.6452, -0.3013, -1.2424,
         -0.3215, -1.4357, -0.5682, -0.5034,  0.7215,  0.1160, -0.1543,  0.1352,
         -0.8648, -0.2945, -0.3526, -0.3892, -0.8371, -1.0664, -1.5108,  0.1204,
         -1.4080, -0.2616, -1.1177, -1.2667,  0.1820, -0.4926,  0.2412,  0.1508,
         -0.7806, -1.5508, -0.1182, -0.7657, -0.3906, -0.1404,  0.0203,  0.2669,
         -0.0567, -0.7683, -1.1666, -1.0871, -1.8775, -0.2152,  0.1756, -2.0806,
         -0.668

TypeError: ResNet.forward() got an unexpected keyword argument 'pseudo_index'

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
import math

# ------------------------------
# 1. 定义 LoRA Linear 层
# ------------------------------
class LoraLinear(nn.Module):
    def __init__(self, base_layer, num_class=1000, rank=8, alpha=16, top_k=1):
        super().__init__()
        self.base_layer = base_layer
        self.num_class = num_class
        self.rank = rank
        self.alpha = alpha
        self.top_k = top_k

        # 冻结原始参数
        for param in base_layer.parameters():
            param.requires_grad = False

        in_features = base_layer.in_features
        out_features = base_layer.out_features

        self.lora_A = nn.Parameter(torch.empty((num_class * rank, in_features)))
        self.lora_B = nn.Parameter(torch.zeros((out_features, num_class * rank)))
        self.scaling = alpha / rank
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))

        self.bias = base_layer.bias
        self.in_features = in_features
        self.out_features = out_features

    def _topk_lora(self, pseudo_index):
        topk_value, topk_index = torch.topk(pseudo_index, self.top_k, dim=1)
        top_index = topk_index.unsqueeze(2).expand(-1, -1, self.rank)
        return top_index.reshape(-1)

    def forward(self, x, pseudo_index=None):
        if pseudo_index is None:
            return self.base_layer(x)
        else:
            orig_output = self.base_layer(x)
            top_index = self._topk_lora(pseudo_index)

            A = self.lora_A[top_index]
            A = A.view(-1, self.top_k * self.rank, A.size(-1))

            B = self.lora_B[:, top_index]
            B = B.view(B.size(0), -1, self.top_k * self.rank)
            B = B.permute(1, 0, 2)

            delta_W = torch.bmm(B, A)
            lora_output = (x.unsqueeze(1) @ delta_W.transpose(1, 2)).squeeze(1) * self.scaling

            return orig_output + lora_output + (self.bias if self.bias is not None else 0)

# ------------------------------
# 2. LoRA Wrapper
# ------------------------------
class Lora(nn.Module):
    def __init__(self, model, rank=8, alpha=16, top_k=1):
        super().__init__()
        self.model = model
        self.rank = rank
        self.alpha = alpha
        self.top_k = top_k
        self._current_pseudo_index = None
        self.hooks = []

        self._apply_lora(self.model)
        self._register_hooks()

    def _apply_lora(self, module):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                lora_linear = LoraLinear(child, num_class=1000, rank=self.rank, alpha=self.alpha, top_k=self.top_k)
                setattr(module, name, lora_linear)
            else:
                self._apply_lora(child)

    def _register_hooks(self):
        for module in self.model.modules():
            if isinstance(module, LoraLinear):
                def hook_fn(mod, inputs, kwargs):
                    if 'pseudo_index' not in kwargs and self._current_pseudo_index is not None:
                        kwargs['pseudo_index'] = self._current_pseudo_index
                    return inputs, kwargs
                handle = module.register_forward_pre_hook(hook_fn, with_kwargs=True)
                self.hooks.append(handle)

    def forward(self, x, *args, **kwargs):
        if 'pseudo_index' not in kwargs:
            with torch.no_grad():
                pseudo_index = self.model(x)
            self._current_pseudo_index = pseudo_index
            return self.model(x, *args, **kwargs)
        else:
            self._current_pseudo_index = kwargs['pseudo_index']
            return self.model(x, *args, **kwargs)

# ------------------------------
# 3. 示例：包装 ResNet18
# ------------------------------
def build_lora_resnet18(rank=8, alpha=16, top_k=1):
    base_model = models.resnet18(pretrained=False)
    return Lora(base_model, rank=rank, alpha=alpha, top_k=top_k)

# ------------------------------
# 4. 测试运行
# ------------------------------
if __name__ == "__main__":
    model = build_lora_resnet18()
    input_x = torch.randn(2, 3, 224, 224)
    output = model(input_x)  # 第一次 forward：生成 pseudo_index
    print("First forward output:", output.shape)

    output2 = model(input_x, pseudo_index=torch.rand(2, 1000))  # 第二次 forward：传入伪标签
    print("Second forward output:", output2.shape)


First forward output: torch.Size([2, 1000])


TypeError: ResNet.forward() got an unexpected keyword argument 'pseudo_index'